In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, random_split
from torchvision import transforms
from sklearn.model_selection import KFold 

* The goal is to train a neural network to approximate an unknown function:
$$ 
f:\mathbb{R}→\mathbb{R} \\
x↦y=f(x) \\
\text{network}(x) \approx f(x)
$$
* As training point, you only have noisy measures from the target function.
$$
\hat{y} = f(x) + noise
$$
* Consider to create a validation set from you training data, or use a k-fold cross-validation strategy. You may find useful these functions from the `scikit-learn` library:
    - [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)
    - [KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html#sklearn.model_selection.KFold) 

In [ ]:
!wget -P regression_dataset https://gitlab.dei.unipd.it/michieli/nnld-2021-22-lab-resources/-/raw/main/homework1/train_data.csv
!wget -P regression_dataset https://gitlab.dei.unipd.it/michieli/nnld-2021-22-lab-resources/-/raw/main/homework1/test_data.csv 

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
trained = True
np.random.seed(0)
torch.manual_seed(0)

### Network Definition

In [ ]:
class Net(nn.Module):
    
    def __init__(self, Ni, Nh1, Nh2, No):
        """
        Ni - Input size
        Nh1 - Neurons in the 1st hidden layer
        Nh2 - Neurons in the 2nd hidden layer
        No - Output size
        """
        super().__init__()

        self.fc1 = nn.Linear(in_features = Ni, out_features = Nh1)
        self.fc2 = nn.Linear(in_features = Nh1, out_features = Nh2)
        self.out = nn.Linear(in_features=Nh2, out_features=No)
        self.act = nn.Sigmoid()
        
    def forward(self, x, additional_out=False):
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        x = self.out(x)
        return x
    

In [ ]:
def train_step(model, train_loader, loss_fn, optimizer, printer=True):
    
    model.train()
    train_loss = []
    for sample_batched in train_loader:
        
        # Move data to device
        x_batch = sample_batched[0].to(device)
        label_batch = sample_batched[1].to(device)
        
        # Forward pass
        out = model(x_batch)

        # Compute loss
        loss = loss_fn(out, label_batch)

        # Backpropagation
        model.zero_grad()
        loss.backward()

        # Update the weights
        optimizer.step()

        # Save train loss for this batch
        loss_batch = loss.detach().cpu().numpy()
        train_loss.append(loss_batch)  
    # Save average train loss over the batches
    train_loss = np.mean(train_loss)
    if(printer): print(f"AVERAGE TRAIN LOSS: {train_loss}")
    return train_loss

    
def validation_step(model, val_loader, loss_fn, printer=True):
    val_loss = []
    
    ### VALIDATION
    model.eval() # Evaluation mode (e.g. disable dropout, batchnorm,...)
    with torch.no_grad(): # Disable gradient tracking
        for sample_batched in val_loader:
            # Move data to device
            x_batch = sample_batched[0].to(device)
            label_batch = sample_batched[1].to(device)

            # Forward pass
            out = model(x_batch)

            # Compute loss
            loss = loss_fn(out, label_batch)

            # Save val loss for this batch
            loss_batch = loss.detach().cpu().numpy()
            val_loss.append(loss_batch)

    # Save average validation loss
    val_loss = np.mean(val_loss)
    if(printer): print(f"AVERAGE VAL LOSS: {val_loss}")
    return val_loss

In [ ]:
# Load from pandas

train_df = pd.read_csv('regression_dataset/train_data.csv')
test_df = pd.read_csv('regression_dataset/test_data.csv')

In [ ]:
# DataSet class 

class CSVDataset(Dataset):

    def __init__(self, file, transform=None):
        self.transform = transform
        self.data = []
        for element in file.index:
            inp = file.iloc[element]['input']
            label = file.iloc[element]['label']
            self.data.append((float(inp), float(label)))
    # Now self.data contains all our dataset.

    def __getitem__(self, idx):
        # Our sample is the element idx of the list self.data        
        sample = self.data[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample
    
    def __len__(self):
        # The length of the dataset is simply the length of the self.data list
        return len(self.data)


In [ ]:
class toTensor(object):
    def __call__(self, sample):
        x,y = sample
        return (torch.tensor([x]).float(), torch.tensor([y]).float())

In [ ]:
composed_transform = transforms.Compose([toTensor()])

train_dataset = CSVDataset(train_df, transform=composed_transform)
test_dataset = CSVDataset(test_df, transform=composed_transform)

In [ ]:
# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Training device: {device}")

### DataLoaders 

In [ ]:
# Train-test splitting for dataset
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset_smpl, val_dataset_smpl = random_split(train_dataset, [train_size, val_size])

# Dataloaders

train_dataloader = DataLoader(train_dataset_smpl, batch_size=int(len(train_dataset_smpl)/2), shuffle=True, num_workers=0)
val_dataloader = DataLoader(val_dataset_smpl, batch_size=len(val_dataset_smpl), shuffle=False, num_workers=0)

test_dataloader  = DataLoader(test_dataset,  batch_size=len(test_dataset), shuffle=False, num_workers=0)

In [ ]:
# Define the loss function
loss_fn = nn.HuberLoss()

In [ ]:
#### TRAINING LOOP

net = Net(1, 256, 128, 1)
# TRAIN!
num_epochs = 500

# Define the optimizer
optimizer = optim.SGD(net.parameters(), lr=1e-3)

net.to(device)
train_loss_log = []
val_loss_log = []
for epoch in range(num_epochs):
    print('#################')
    print(f'# EPOCH {epoch}')
    print('#################')
    t_loss = train_step(net, train_dataloader, loss_fn, optimizer)
    train_loss_log.append(t_loss)
    v_loss = validation_step(net, val_dataloader, loss_fn)
    val_loss_log.append(v_loss)


In [ ]:
# Plot losses
plt.figure(figsize=(12,8))
plt.semilogy(train_loss_log, label='Train loss')
plt.semilogy(val_loss_log, label='Validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid()
plt.legend()
#plt.savefig('simple_losses.pdf', format = 'pdf')
plt.show()

### Access network parameters

In [ ]:
# First hidden layer
h1_w = net.fc1.weight.data.cpu().numpy()
h1_b = net.fc1.bias.data.cpu().numpy()

# Second hidden layer
h2_w = net.fc2.weight.data.cpu().numpy()
h2_b = net.fc2.bias.data.cpu().numpy()

# Output layer
out_w = net.out.weight.data.cpu().numpy()
out_b = net.out.bias.data.cpu().numpy()

### Weights histogram

In [ ]:
# Weights histogram
fig, axs = plt.subplots(3, 1, figsize=(12,8))
axs[0].hist(h1_w.flatten(), 50)
axs[0].set_title('First hidden layer weights')
axs[1].hist(h2_w.flatten(), 50)
axs[1].set_title('Second hidden layer weights')
axs[2].hist(out_w.flatten(), 50)
axs[2].set_title('Output layer weights')
[ax.grid() for ax in axs]
plt.tight_layout()
plt.show()

### Testing

In [ ]:
for sample_batched in test_dataloader:
    # Move data to device
    x_batch = sample_batched[0].to(device)
    label_batch = sample_batched[1].to(device)

    net.eval()
    with torch.no_grad(): # turn off gradients computation
        y_vec = net(x_batch)
        error = nn.functional.mse_loss(y_vec, label_batch) #Compute the mean squared error
        
        y_vec = y_vec.cpu().numpy()
        label_batch = label_batch.cpu().numpy()
        # Plot output
        plt.figure(figsize=(12,8))
        plt.plot(y_vec, label='Network output')
        plt.plot(label_batch, label='True model')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.grid()
        plt.legend()
        #plt.savefig('simple_prediction.pdf', format = 'pdf', bbox_inches = 'tight')
        plt.show()
        
        
print('#################')
print(f'# MSE {error}')
print('#################')

### Advanced desing

In [ ]:
class Net_v2(nn.Module):
    
    def __init__(self, Ni, Nh1, Nh2, No):
        """
        Ni - Input size
        Nh1 - Neurons in the 1st hidden layer
        Nh2 - Neurons in the 2nd hidden layer
        No - Output size
        """
        super().__init__()
        self.fc1 = nn.Linear(in_features = Ni, out_features = Nh1)
        self.fc2 = nn.Linear(in_features = Nh1, out_features = Nh2)
        self.out = nn.Linear(in_features=Nh2, out_features=No)
        self.act = nn.ELU() 
        print("Network initialized!")
        
    def forward(self, x, additional_out=False):
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        x = self.out(x)
        return x
    

In [ ]:
if not trained:
    ###########OPTIMIZATION###########
    ! pip install optuna
    import optuna
    from optuna.integration import PyTorchLightningPruningCallback
    EPOCHS = 250

    train_loss_log = []
    val_loss_log = []

    def objective(trial):

        # Number of units for hidden layer
        output_dims = [trial.suggest_int("n_units_l{}".format(i), 64, 256, log=True) for i in range(2)]
        # Learning rate (log)
        learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
        # Weight decay 
        w_decay = trial.suggest_float("weight_decay", 0, 1e-7)

        # Delta of loss (threshold to change between delta-scaled L1 and L2 loss)
        delta = trial.suggest_float("delta", 0, 2)

        model = Net_v2(1, output_dims[0], output_dims[1], 1).to(device)
        optimizer_name = "Adam"
        optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=learning_rate, weight_decay=w_decay)

        loss_func = nn.HuberLoss(delta=delta)

        for epoch in range(EPOCHS):
            t_loss = train_step(model, train_dataloader, loss_fn, optimizer, printer = False)
            train_loss_log.append(t_loss)
            v_loss = validation_step(model, val_dataloader, loss_fn, printer = False)
            val_loss_log.append(v_loss)

            trial.report(val_loss_log[-1], epoch)

            # Handle pruning based on the intermediate value.
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        return val_loss_log[-1]

    pruner: optuna.pruners.BasePruner = optuna.pruners.MedianPruner(n_startup_trials=3, n_warmup_steps=10, interval_steps=1)

    study = optuna.create_study(study_name="Regression", direction="minimize", pruner=pruner)
    study.optimize(objective, n_trials=1000)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [ ]:
print('########## BEST TRIAL ########## \n')
print('Params:\n')
print("Number of units first hidden layer: 163")
print("Number of units second hidden layer: 150")
print('learning rate: 0.008868384223793775')
print('weight decay: 4.896378094756895e-08')
print('delta: 1.0377865474636057')

In [ ]:
net2 = Net_v2(1, 163, 150, 1)
net2.to(device)
optimizer = optim.Adam(net2.parameters(), lr=0.008868384223793775, weight_decay=4.896378094756895e-08)
loss_fn = nn.HuberLoss(delta=1.0377865474636057)

### Network weights before training

In [ ]:

h1_w = net2.fc1.weight.data.cpu().numpy()
h1_b = net2.fc1.bias.data.cpu().numpy()

h2_w = net2.fc2.weight.data.cpu().numpy()
h2_b = net2.fc2.bias.data.cpu().numpy()

out_w = net2.out.weight.data.cpu().numpy()
out_b = net2.out.bias.data.cpu().numpy()

# Weights histogram
fig, axs = plt.subplots(3, 1, figsize=(12,8))
axs[0].hist(h1_w.flatten(), 50)
axs[0].set_title('First hidden layer weights', fontsize = 18)
axs[1].hist(h2_w.flatten(), 50)
axs[1].set_title('Second hidden layer weights', fontsize = 18)
axs[2].hist(out_w.flatten(), 50)
axs[2].set_title('Output layer weights', fontsize = 18)
[ax.grid() for ax in axs]
plt.tight_layout()
#plt.savefig('Weights_before.pdf', format = 'pdf', bbox_inches = 'tight') 
plt.show()

In [ ]:
## KFold split 
k=10
splits=KFold(n_splits=k,shuffle=True,random_state=0)

### Training

In [ ]:
if not trained:
    ######## TRAIN ########
    train_loss_log = []
    val_loss_log = []
    best_loss = np.infty
    num_epochs = 300
    patience = 15

    for epoch in range(num_epochs):
        print('#################')
        print(f'# EPOCH {epoch}')
        print('#################')

        train_fold = []
        test_fold = []

        # Train the model for each fold
        for train_idx,val_idx in splits.split(train_dataset):

            train_sampler = SubsetRandomSampler(train_idx)
            test_sampler = SubsetRandomSampler(val_idx)
            train_loader = DataLoader(train_dataset, batch_size=len(train_sampler), sampler=train_sampler)
            test_loader = DataLoader(train_dataset, batch_size=len(test_sampler), sampler=test_sampler)

            t_loss = train_step(net2, train_loader, loss_fn, optimizer, printer=False)
            v_loss = validation_step(net2, test_loader, loss_fn, printer=False)

            train_fold.append(t_loss)
            test_fold.append(v_loss)
        # Average the performances of the fold
        train_loss_log.append(np.mean(train_fold))
        print(f"AVERAGE TRAIN LOSS: {train_loss_log[-1]}")
        val_loss_log.append(np.mean(test_fold))
        print(f"AVERAGE VAL LOSS: {val_loss_log[-1]}")

        # Implement early stopping
        if(val_loss_log[-1] < best_loss):
            best_loss = val_loss_log[-1]
            patience = 15
        else:
            patience -= 1
            if(patience == 0): 
                print("#################\nLearning stopped because the validation error was not improving\n#################")
                break 
                
    #save the model               
    net_state_dict = net2.state_dict()
    print(net_state_dict.keys())
    # Save the state dict to a file
    torch.save(net_state_dict, 'regressor_v3.torch')
else:
    # Load the state dict previously saved
    net_state_dict = torch.load('regressor_v3.torch')
    # Update the network parameters
    net2.load_state_dict(net_state_dict)

In [ ]:
if not trained:
    # Plot losses
    plt.figure(figsize=(12,8))
    plt.semilogy(train_loss_log, label='Train loss')
    plt.semilogy(val_loss_log, label='Validation loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid()
    plt.legend()
    #plt.savefig('loss_regression.pdf', format = 'pdf', bbox_inches = 'tight') 
    plt.show()

### Network weights after training

In [ ]:
h1_w = net2.fc1.weight.data.cpu().numpy()
h1_b = net2.fc1.bias.data.cpu().numpy()

h2_w = net2.fc2.weight.data.cpu().numpy()
h2_b = net2.fc2.bias.data.cpu().numpy()

out_w = net2.out.weight.data.cpu().numpy()
out_b = net2.out.bias.data.cpu().numpy()

# Weights histogram
fig, axs = plt.subplots(3, 1, figsize=(12,8))
axs[0].hist(h1_w.flatten(), 50)
axs[0].set_title('First hidden layer weights', fontsize = 18)
axs[1].hist(h2_w.flatten(), 50)
axs[1].set_title('Second hidden layer weights', fontsize = 18)
axs[2].hist(out_w.flatten(), 50)
axs[2].set_title('Output layer weights', fontsize = 18)
[ax.grid() for ax in axs]
plt.tight_layout()
#plt.savefig('weights_after_regression.pdf', format = 'pdf', bbox_inches = 'tight') 
plt.show()

### Activation profile

In [ ]:
def get_activation(layer, input, output):
    global activation
    activation = torch.nn.functional.elu(output)

### Register hook  
hook_handle_1 = net2.fc1.register_forward_hook(get_activation)

### Analyze activations
net2 = net2.to(device)
net2.eval()
with torch.no_grad():
    x1 = torch.tensor([10]).float().to(device)
    y1 = net2(x1)
    z1 = activation
    x2 = torch.tensor([-100]).float().to(device)
    y2 = net2(x2)
    z2 = activation
    x3 = torch.tensor([200]).float().to(device)
    y3 = net2(x3)
    z3 = activation

### Remove hook
hook_handle_1.remove()

### Plot activations
fig, axs = plt.subplots(3, 1, figsize=(12,6))
axs[0].stem(z1.cpu().numpy(), use_line_collection=True)
axs[0].set_title('First layer activations for input x=%.2f' % x1, fontsize = 18)
axs[1].stem(z2.cpu().numpy(), use_line_collection=True)
axs[1].set_title('First layer activations for input x=%.2f' % x2, fontsize = 18)
axs[2].stem(z3.cpu().numpy(), use_line_collection=True)
axs[2].set_title('First layer activations for input x=%.2f' % x3, fontsize = 18)
plt.tight_layout()
#plt.savefig('activation_profile_h1.pdf', format = 'pdf', bbox_inches = 'tight') 
plt.show()

hook_handle_2 = net2.fc2.register_forward_hook(get_activation)

### Analyze activations
net2 = net2.to(device)
net2.eval()
with torch.no_grad():
    x1 = torch.tensor([10]).float().to(device)
    y1 = net2(x1)
    z1 = activation
    x2 = torch.tensor([-100]).float().to(device)
    y2 = net2(x2)
    z2 = activation
    x3 = torch.tensor([200]).float().to(device)
    y3 = net2(x3)
    z3 = activation

### Remove hook
hook_handle_2.remove()

fig, axs = plt.subplots(3, 1, figsize=(12,6))
axs[0].stem(z1.cpu().numpy(), use_line_collection=True)
axs[0].set_title('Last layer activations for input x=%.2f' % x1, fontsize = 18)
axs[1].stem(z2.cpu().numpy(), use_line_collection=True)
axs[1].set_title('Last layer activations for input x=%.2f' % x2, fontsize = 18)
axs[2].stem(z3.cpu().numpy(), use_line_collection=True)
axs[2].set_title('Last layer activations for input x=%.2f' % x3, fontsize = 18)
plt.tight_layout()
#plt.savefig('activation_profile_h2.pdf', format = 'pdf', bbox_inches = 'tight') 
plt.show()

### Testing

In [ ]:
for sample_batched in test_dataloader:
    # Move data to device
    x_batch = sample_batched[0].to(device)
    label_batch = sample_batched[1].to(device)

    net2.eval()
    with torch.no_grad(): # turn off gradients computation
        y_vec = net2(x_batch)
        error = nn.functional.mse_loss(y_vec, label_batch) #Compute the mean squared error
        
        y_vec = y_vec.cpu().numpy()
        label_batch = label_batch.cpu().numpy()
        # Plot output
        plt.figure(figsize=(12,8))
        plt.plot(y_vec, label='Network output')
        plt.plot(label_batch, label='True model')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.grid()
        plt.legend()
        #plt.savefig('result_regression.pdf', format = 'pdf', bbox_inches = 'tight') 
        plt.show()
        
        
print('#################')
print(f'# MSE {error}')
print('#################')